In [25]:
import multiprocessing as mp
import psutil
import pickle
import time
import sys

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !xz -d -v Moscow_mkad.osm.xz

Moscow_mkad.osm.xz (1/1)
xz: Moscow_mkad.osm.xz: No such file or directory


In [26]:
# парсинг XML
import xml.etree.ElementTree as ET
tree = ET.parse('./Moscow_mkad.osm')
root=tree.getroot()
element = root[0]

In [27]:
"""
Создание матрицы высота
Минимум: 75
Максимум: 317
Генерация np матрицы высот
"""
cell_size = 0.00083333333333333
lat_start = 55.0
lon_start = 35.0

matrix_of_heights = []
with open('./srtm_44_01.asc') as f:
    data_str = f.readlines()

for row in data_str[6:]:
    row_list = list(map(int, row.split()))
    matrix_of_heights.append(row_list)

del row_list
del data_str

matrix_of_heights = np.array(matrix_of_heights, np.ushort)


In [28]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367 * c * 1000
    return m


def get_height(lat: float, lon: float) -> float:
    """
    Получение высоты над уровнем моря по координатам
    :param lat: Долгота точки
    :param lon: Широта точки
    :return: Высота над уровнем моря
    """
    semi_x = (lat - lat_start) / cell_size
    semi_y = (lon - lon_start) / cell_size
    if semi_x > 6000 or semi_y > 6000:
        raise Exception('Out of bounds, not in Moscow')

    points = [matrix_of_heights[int(np.floor(semi_x)), int(np.floor(semi_y))],
              matrix_of_heights[int(np.floor(semi_x)), int(np.ceil(semi_y))],
              matrix_of_heights[int(np.ceil(semi_x)), int(np.floor(semi_y))],
              matrix_of_heights[int(np.ceil(semi_x)), int(np.ceil(semi_y))]]
    floor_lat = np.floor(semi_x) * cell_size + lat_start
    floor_lon = np.floor(semi_y) * cell_size + lon_start
    ceil_lat = np.ceil(semi_x) * cell_size + lat_start
    ceil_lon = np.ceil(semi_y) * cell_size + lon_start
    coordinates = [[floor_lat, floor_lon], [floor_lat, ceil_lon], [ceil_lat, floor_lon],
                   [ceil_lat, ceil_lon]]
    idx_min, min_ = 0, 200
    for idx, point in enumerate(coordinates):
        dist_ = haversine_np(lon, lat, point[1], point[0])
        if dist_ < min_:
            min_ = dist_
            idx_min = idx
    triangle = [points[idx_min-1], points[idx_min], points[(idx_min+1)%4]]
    return sum(triangle)/3


get_height(55.7558, 37.6173)

157.66666666666666

In [29]:
"""
 Генерация листа эджей формата 
 [{"nodes": [], "highway":"", "surface":"asphalt", "lanes":1, "width":2}]
 А также листа нод формата [{id, lat, lon}]
"""
nodes = []
edges = []
for i in tqdm(range(1, len(root))):
    obj = root[i]
    if obj.tag == 'node':
        nodes.append({'id': obj.attrib['id'], 
                      'lat': obj.attrib['lat'], 'lon': obj.attrib['lon']})
    if obj.tag == 'way':
        tmp = {"nodes": [], "highway":"", "surface":"asphalt", "lanes":1, 
               "width":2}
        ok = False
        for child in obj:
            if child.tag == 'nd':
                tmp['nodes'].append(child.attrib["ref"])
            if child.tag == 'tag':
              if child.attrib['k'] == 'highway':
                ok = True
              if child.attrib['k'] in tmp:
                  tmp[child.attrib['k']] = child.attrib['v']
        if ok and tmp['highway'] in ['pedestrian', 'bridleway', 'cycleway', 
                                     'footway', 'living_street', 'path', 
                                     'steps', 'residential', 'service']:
            edges.append(tmp)

print(f'Edges: {len(edges):,}, Nodes: {len(nodes):,}')


Edges: 250,642, Nodes: 3,269,157


In [7]:
# Удаления парсера XML
del tree
del root

In [38]:
# словарь айди_ноды: её координаты
id2cor = {}

for i in nodes:
    id2cor[int(i['id'])] = {'lat': float(i['lat']), 'lon': float(i['lon'])}

In [31]:
# Генерация листа словарей edge
# Медленная!
count = 0
edge_list = []
for edge in tqdm(edges):
    for i in range(len(edge['nodes'])-1):
      node_1 = id2cor[edge['nodes'][i]]
      node_2 = id2cor[edge['nodes'][i+1]]
      lon1 = float(node_1['lon'])
      lat1 = float(node_1['lat'])
      lon2 = float(node_2['lon'])
      lat2 = float(node_2['lat']) 
      dist = haversine_np(lon1, lat1, lon2, lat2)
      hight1 = get_height(lat1, lon1)
      hight2 = get_height(lat2, lon2)
      if isinstance(edge['width'], str):
        try:
          width_f = float(edge['width'])
        except ValueError:
          width_f = 1
      else:
        width_f = float(edge['width'])

      edge_list.append({"edge_id":count, "id1":int(edge['nodes'][i]), 
                        "id2":int(edge['nodes'][i+1]), "dist": dist, 
                        "highway": edge['highway'], "surface":edge['surface'], 
                        "lanes": int(edge['lanes']), "width":width_f, 
                        'hight1': hight1, 'hight2': hight2})
      count += 1
  
edge_list[22:25]

[{'edge_id': 22,
  'id1': 5361369319,
  'id2': 5170048255,
  'dist': 67.93649407217399,
  'highway': 'residential',
  'surface': 'asphalt',
  'lanes': 1,
  'width': 2.0,
  'hight1': 110.0,
  'hight2': 110.0},
 {'edge_id': 23,
  'id1': 5170048255,
  'id2': 5361369240,
  'dist': 188.34466560928217,
  'highway': 'residential',
  'surface': 'asphalt',
  'lanes': 1,
  'width': 2.0,
  'hight1': 110.0,
  'hight2': 110.33333333333333},
 {'edge_id': 24,
  'id1': 5361369240,
  'id2': 5361369238,
  'dist': 68.11161372538463,
  'highway': 'residential',
  'surface': 'asphalt',
  'lanes': 1,
  'width': 2.0,
  'hight1': 110.33333333333333,
  'hight2': 111.0}]

In [32]:
import pandas as pd
graph = pd.DataFrame(edge_list, columns=edge_list[0].keys())
graph.to_csv('edge_list.csv', index=False)

graph.head()

,edge_id,id1,id2,dist,highway,surface,lanes,width,hight1,hight2
0,0,858251137,1347901221,42.742844,footway,asphalt,1,2.0,109.000000,109.666667
1,1,1347901221,1416377962,9.152257,footway,asphalt,1,2.0,109.666667,109.666667
2,2,1416377962,1347901220,20.770940,footway,asphalt,1,2.0,109.666667,109.333333
3,3,1347901220,1854100545,7.954432,footway,asphalt,1,2.0,109.333333,109.333333
4,4,1854100545,1347901219,76.984630,footway,asphalt,1,2.0,109.333333,109.666667


In [39]:
# Генерация локации еджа
edge_cors = []
for i in tqdm(range(len(graph))):
    cor1 = np.array([id2cor[graph.id1.iloc[i]]['lat'], id2cor[graph.id1.iloc[i]]['lon']])
    cor2 = np.array([id2cor[graph.id2.iloc[i]]['lat'], id2cor[graph.id2.iloc[i]]['lon']])
    cor = (np.sum([cor1, cor2], axis=0)) / 2
    edge_cors.append(cor)

In [41]:
edge_cors[:5]

[array([55.8832217, 37.460129 ]),
 array([55.8832634 , 37.45972165]),
 array([55.88319405, 37.45953315]),
 array([55.88310695, 37.45936455]),
 array([55.8827412, 37.4593314])]

In [44]:
# Добавление локации еджа в граф
graph["lat"] = np.array(edge_cors)[:, 0]
graph["lon"] = np.array(edge_cors)[:, 1]

graph.head()

,edge_id,id1,id2,dist,highway,surface,lanes,width,hight1,hight2,lat,lon
0,0,858251137,1347901221,42.742844,footway,asphalt,1,2.0,109.000000,109.666667,55.883222,37.460129
1,1,1347901221,1416377962,9.152257,footway,asphalt,1,2.0,109.666667,109.666667,55.883263,37.459722
2,2,1416377962,1347901220,20.770940,footway,asphalt,1,2.0,109.666667,109.333333,55.883194,37.459533
3,3,1347901220,1854100545,7.954432,footway,asphalt,1,2.0,109.333333,109.333333,55.883107,37.459365
4,4,1854100545,1347901219,76.984630,footway,asphalt,1,2.0,109.333333,109.666667,55.882741,37.459331


In [43]:
graph.to_csv("edge_with_location.csv")

In [11]:
tmp_graph = graph.drop(columns=['highway', 'surface', 'width', 'hight1', 'hight2', 'dist', 'lanes'])
graph_np = tmp_graph.values
del tmp_graph

graph_np[:2], len(graph_np)

(array([[         0,  858251137, 1347901221],
        [         1, 1347901221, 1416377962]]),
 992193)

In [3]:
# uint
# max id: 8912249766
# min id: 27717690
fast_graph_np = np.genfromtxt('fast_graph.csv', dtype=np.uint, delimiter=',')[1:]
# fast_graph_np = []
# for edge in tqdm(graph_np):
#   fast_graph_np.append([edge[0], int(edge[1]), int(edge[2])])

# fast_graph_np = np.array(fast_graph_np, dtype=np.uint)
fast_graph_np[:5]

array([[         0,  858251137, 1347901221],
       [         1, 1347901221, 1416377962],
       [         2, 1416377962, 1347901220],
       [         3, 1347901220, 1854100545],
       [         4, 1854100545, 1347901219]], dtype=uint64)

In [1]:
fast_graph_df = pd.DataFrame(data=fast_graph_np)
fast_graph_df.to_csv('fast_graph.csv', index=False)
fast_graph_np[:,1].min(), fast_graph_np[:,1].max(), fast_graph_np[:,2].min(), fast_graph_np[:,2].max()

NameError: name 'pd' is not defined

In [5]:
from tqdm import tqdm
size = len(fast_graph_np)+1
workers_count = mp.cpu_count() - 10
part_size = len(fast_graph_np[:size])//workers_count

workers_count
print(part_size, mp.cpu_count(), workers_count, len(fast_graph_np[:size]))


def spawn():
    dataframe_list = []
    procs = list()
    
    manager = mp.Manager()
    return_list = manager.list()
    
    for cpu in range(workers_count):
        up_border = part_size*(cpu+1)
        p = mp.Process(target=run_child, args=(up_border, return_list))
        p.start()
        procs.append(p)
    for idx, p in enumerate(procs):
        p.join()
        print('Done: {}'.format(idx), end=' ')
    
    return return_list

return_list = spawn()

11537 96 86 992193


  3%|▎         | 320/11537 [00:28<25:39,  7.29it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  7%|▋         | 836/11537 [01:26<20:24,  8.74it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  8%|▊         | 940/11537 [01:33<14:56, 11.81it/s]]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.io

Done: 0 Done: 1 Done: 2 Done: 3 Done: 4 Done: 5 Done: 6 Done: 7 Done: 8 Done: 9 Done: 10 Done: 11 Done: 12 Done: 13 Done: 14 Done: 15 Done: 16 

100%|██████████| 11537/11537 [24:18<00:00,  7.91it/s] 


Done: 17 Done: 18 Done: 19 Done: 20 Done: 21 Done: 22 Done: 23 Done: 24 Done: 25 Done: 26 Done: 27 Done: 28 Done: 29 Done: 30 Done: 31 Done: 32 Done: 33 Done: 34 Done: 35 Done: 36 Done: 37 Done: 38 Done: 39 Done: 40 Done: 41 Done: 42 Done: 43 Done: 44 Done: 45 Done: 46 Done: 47 Done: 48 Done: 49 Done: 50 Done: 51 Done: 52 Done: 53 Done: 54 Done: 55 Done: 56 Done: 57 Done: 58 Done: 59 Done: 60 Done: 61 Done: 62 Done: 63 Done: 64 Done: 65 Done: 66 Done: 67 Done: 68 Done: 69 Done: 70 Done: 71 Done: 72 Done: 73 Done: 74 Done: 75 Done: 76 Done: 77 Done: 78 Done: 79 Done: 80 Done: 81 Done: 82 Done: 83 Done: 84 Done: 85 

In [4]:
def run_child(up_border, return_list):
#     print('Run child: ', up_border, end='\t')
    adj = []

    for i in tqdm(range(up_border - part_size, up_border)):
        a = fast_graph_np[fast_graph_np[:,2] == fast_graph_np[i,1]][:, 0]
        b = fast_graph_np[fast_graph_np[:,1] == fast_graph_np[i,1]][:, 0].tolist()    
        b.remove(i)
        df = b + a.tolist()    
        adj.append(df)

        
    return_list.extend(adj)


In [6]:
adj = list(return_list) 
len(adj)

992182

In [13]:
# import pickle

# with open('adj.pkl', 'wb') as fp:
#     pickle.dump(adj, fp, protocol=pickle.HIGHEST_PROTOCOL)
adj[:10]

[[311498],
 [311499],
 [311500],
 [311501, 645490],
 [311480, 311502],
 [311503],
 [307926, 307925, 311504],
 [934535],
 [934687, 311506, 515208],
 [934629, 311507, 934628]]

In [10]:
# adj_str = cast_to_string(return_list)
adj_df = pd.DataFrame(data=adj)

In [11]:
adj_df.to_csv('duo_edge.csv', index=False)

In [7]:
def cast_to_string(routes_property):
    return [str(int).replace("[", "").replace("]", "") for int in routes_property]

In [ ]:
adj_str = cast_to_string(adj)
adj_df = pd.DataFrame(adj_str, columns = ["adjacent"])

In [ ]:
import pickle

with open('nodes.pkl', 'wb') as fp:
    pickle.dump(nodes, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

with open('edges.pkl', 'wb') as fp:
    pickle.dump(edges, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
count_pos = 0
count_neg = 0
edge_list = []
for edge in edges:
    tmp = []
    for node in edge:
        if node in id2cor:
            count_pos += 1
            tmp.append(node)
        else:
            count_neg += 1
    edge_list.append(tmp)

In [ ]:
count_pos

1636584

In [ ]:
a = []
for edge in edges[102]:
    a.append({"id": edge, "lat": id2cor[edge]['lat'], "lon": id2cor[edge]['lon']})

In [ ]:
df = pd.DataFrame(a, columns=["id", "lat", 'lon'])
df.to_csv('list2.csv', index=False)

In [ ]:
dist_edge = []
for edge in tqdm(edges):
    for i in range(len(edge)-1):
        dist = haversine_np(float(id2cor[edge[i]]['lon']), float(id2cor[edge[i]]['lat']), float(id2cor[edge[i + 1]]['lon']), float(id2cor[edge[i + 1]]['lat']))
        dist_edge.append({"id1": edge[i], "id2": edge[i + 1], 'd': dist * 1000})

KeyError: ignored

In [ ]:
import pickle

with open('edge_dist.pkl', 'wb') as fp:
    pickle.dump(dist_edge, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
edge_list_for_kepler = []
for edge in tqdm(edges):
    for i in range(len(edge['nodes'])-1):
        node_1 = id2cor[edge['nodes'][i]]
        node_2 = id2cor[edge['nodes'][i+1]]
        lon1 = float(node_1['lon'])
        lat1 = float(node_1['lat'])
        lon2 = float(node_2['lon'])
        lat2 = float(node_2['lat']) 
        dist = haversine_np(lon1, lat1, lon2, lat2)
        hight1 = get_height(lat1, lon1)
        hight2 = get_height(lat2, lon2)
        if isinstance(edge['width'], str):
            try:
              width_f = float(edge['width'])
            except ValueError:
              width_f = 1
        else:
            width_f = float(edge['width'])

        edge_list_for_kepler.append({"dist": dist, "highway": edge['highway'], "surface":edge['surface'], 
                                   "lanes": int(edge['lanes']), "width":edge['width'], 
                                   'hight1': hight1, 'hight2': hight2, 'lat1': lat1, 'lon1': lon1, 
                                   'lat2': lat2, 'lon2': lon2})

100%|██████████| 250642/250642 [02:07<00:00, 1962.75it/s]


In [23]:
df = pd.DataFrame(edge_list_for_kepler, columns=edge_list_for_kepler[0].keys())
df.to_csv('edge_list_for_kepler.csv', index=False)

In [ ]:
import pickle

with open('nodes.p', 'rb') as fp:
    nodes = pickle.load(fp)

EOFError: Ran out of input

In [ ]:
edges_pickles = pickle.load(open('nodes.p', 'rb'))

In [ ]:
pd.DataFrame(hist_nodes).to_csv("hist_nodes.csv")

In [ ]:
import json
with open('dtp_moskva.geojson') as fp:
    data = json.load(fp)

In [ ]:
data['features'][1]['geometry']

{'type': 'Point', 'coordinates': [37.489332, 55.841157]}

In [ ]:
len(data['features'])

57130

In [ ]:
dtp = []
for i in tqdm(range(len(data['features']))):
    dtp.append({"lat": data['features'][i]['geometry']['coordinates'][1], "lon":data['features'][i]['geometry']['coordinates'][0]})